In [ ]:
import os
from dotenv import dotenv_values

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, PipelinePromptTemplate

# Get API key from environment variables
config = dotenv_values(".env")
OPEN_AI_API_KEY = config["OPEN_AI_API_KEY"]

# Check if API key exists
if not OPEN_AI_API_KEY:
    raise ValueError(
        "OPENAI_API_KEY not found in environment variables. Please add it to your .env file.")


config = dotenv_values(".env")
OPEN_AI_API_KEY = config["OPEN_AI_API_KEY"]

model = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=OPEN_AI_API_KEY
)


In [ ]:
print("Create Templates")

code_template = PromptTemplate(
    input_variables=["language", "task"],
    template="Write a vary short {language} function that will complete the following task: {task}: only return the function declaration code, no other text"
)

test_template = PromptTemplate(
    input_variables=["language", "code"],
    template="Write a very short unit test for the following {code} using {language} programming language. Only return the test code, no other text"
)

instruction_template = PromptTemplate(
    input_variables=["language", "code", "test"],
    template="I will provide you with a {language} function and its test. Please give brief instructions on how to implement and run them:\n\nFunction:\n{code}\n\nTest:\n{test}\n\nProvide clear, concise instructions."
)

In [ ]:
# Define the pipeline
print("Define the pipelines")

pipeline_prompts = [
    ("code", code_template),
    ("test", test_template),
    ("instructions", instruction_template)
]

# Initialize input dictionary with initial parameters
my_input = {
    "language": "Python",
    "task": "Create a function that returns the surface area of a square."
}

In [ ]:
print("Process each prompt")

results = {}
for name, prompt in pipeline_prompts:
    # Get the response from the model for each prompt
    prompt_response = model.invoke(prompt.invoke(my_input).to_string())
    results[name] = prompt_response.content
    
    # Update input dictionary for next prompt
    my_input[name] = prompt_response.content

    # Here you could add your database saving logic, for example:
    # save_to_database(name, prompt_response.content)

In [ ]:
# Get final response from the model
print("Getting Final Response...")
response = model.invoke(my_input["instructions"])


print("Response")
print(response.content)